In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

feat_no = '112_eda_'

In [14]:
import datetime
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

clf_pred = utils.read_pkl_gzip('../stack/1225_102_outlier_classify_lgb_auc0-88940189917734_83features.gz').set_index(key)
clf_pred.columns = [f"clf_{col}" for col in clf_pred.columns]
out_pred = utils.read_pkl_gzip('../stack/1225_172_en_route_lgb_CV1-55583888596708_83features.gz').set_index(key)
out_pred.columns = [f"out_{col}" for col in out_pred.columns]
best_pred = utils.read_pkl_gzip('../stack/1225_182_en_route_lgb_CV3-65604336032456_83features.gz').set_index(key)
best_pred.columns = [f"best_{col}" for col in best_pred.columns]

best_pred[clf_pred.columns] = clf_pred
best_pred[out_pred.columns] = out_pred


In [18]:
import glob

win_path = '../features/4_winner/*'
# Data Load
base = utils.read_df_pkl('../input/base*')
win_path_list = glob.glob(win_path)
train_path_list = []
test_path_list = []
for path in win_path_list:
    if path.count('train'):
        train_path_list.append(path)
    elif path.count('test'):
        test_path_list.append(path)

base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)
train_feature_list = utils.parallel_load_data(path_list=train_path_list)
test_feature_list = utils.parallel_load_data(path_list=test_path_list)
train = pd.concat(train_feature_list, axis=1)
train = pd.concat([base_train, train], axis=1)
test = pd.concat(test_feature_list, axis=1)
test = pd.concat([base_test, test], axis=1)

train_test = pd.concat([train, test], ignore_index=True)
train_test.head()

100%|██████████| 3/3 [00:00<00:00, 70.96it/s]


,111_ker_card_id_total@,111_ker_dayofweek@,111_ker_elapsed_time@,111_ker_feature_1@,111_ker_feature_2@,111_ker_feature_3@,111_ker_hist_authorized_flag_mean@,111_ker_hist_authorized_flag_sum@,111_ker_hist_card_id_size@,111_ker_hist_category_1_mean@,111_ker_hist_category_1_sum@,111_ker_hist_category_2_mean_mean@,111_ker_hist_category_3_mean_mean@,111_ker_hist_dayofweek_nunique@,111_ker_hist_first_buy@,111_ker_hist_hour_nunique@,111_ker_hist_installments_max@,111_ker_hist_installments_mean@,111_ker_hist_installments_min@,111_ker_hist_installments_sum@,111_ker_hist_installments_var@,111_ker_hist_merchant_category_id_nunique@,111_ker_hist_merchant_id_nunique@,111_ker_hist_month_diff_mean@,111_ker_hist_month_lag_max@,111_ker_hist_month_lag_mean@,111_ker_hist_month_lag_min@,111_ker_hist_month_lag_var@,111_ker_hist_month_nunique@,111_ker_hist_purchase_amount_max@,111_ker_hist_purchase_amount_mean@,111_ker_hist_purchase_amount_min@,111_ker_hist_purchase_amount_sum@,111_ker_hist_purchase_amount_var@,111_ker_hist_purchase_date_average@,111_ker_hist_purchase_date_diff@,111_ker_hist_purchase_date_max@,111_ker_hist_purchase_date_min@,111_ker_hist_purchase_date_uptonow@,111_ker_hist_subsector_id_nunique@,111_ker_hist_weekend_mean@,111_ker_hist_weekend_sum@,111_ker_hist_weekofyear_nunique@,111_ker_hist_year_nunique@,111_ker_month@,111_ker_new_hist_card_id_size@,111_ker_new_hist_category_1_mean@,111_ker_new_hist_category_1_sum@,111_ker_new_hist_category_2_mean_mean@,111_ker_new_hist_category_3_mean_mean@,111_ker_new_hist_dayofweek_nunique@,111_ker_new_hist_first_buy@,111_ker_new_hist_hour_nunique@,111_ker_new_hist_installments_max@,111_ker_new_hist_installments_mean@,111_ker_new_hist_installments_min@,111_ker_new_hist_installments_sum@,111_ker_new_hist_installments_var@,111_ker_new_hist_merchant_category_id_nunique@,111_ker_new_hist_merchant_id_nunique@,111_ker_new_hist_month_diff_mean@,111_ker_new_hist_month_lag_max@,111_ker_new_hist_month_lag_mean@,111_ker_new_hist_month_lag_min@,111_ker_new_hist_month_lag_var@,111_ker_new_hist_month_nunique@,111_ker_new_hist_purchase_amount_max@,111_ker_new_hist_purchase_amount_mean@,111_ker_new_hist_purchase_amount_min@,111_ker_new_hist_purchase_amount_sum@,111_ker_new_hist_purchase_amount_var@,111_ker_new_hist_purchase_date_average@,111_ker_new_hist_purchase_date_diff@,111_ker_new_hist_purchase_date_max@,111_ker_new_hist_purchase_date_min@,111_ker_new_hist_purchase_date_uptonow@,111_ker_new_hist_subsector_id_nunique@,111_ker_new_hist_weekend_mean@,111_ker_new_hist_weekend_sum@,111_ker_new_hist_weekofyear_nunique@,111_ker_new_hist_year_nunique@,111_ker_purchase_amount_total@,111_ker_weekofyear@,card_id,target
0,283.0,3.0,572.0,0.013145,0.008752,0.011428,0.950000,247,260,0.000000,0,0.072502,0.346719,7,26.0,23,1,0.015385,0,4,0.015206,41,94,10.000000,0,-3.911538,-8,5.748901,9,2.258395,-0.638341,-0.739395,-165.968739,0.045003,0.930769,242,1.519551e+09,1.498573e+09,302,21,0.346154,90,35,2,6.0,23.0,0.000000,0.0,-0.550160,-0.592993,7.0,277.0,8.0,0.0,0.000000,0.0,0.0,0.000000,14.0,23.0,9.869565,2.0,1.478261,1.0,0.260870,2.0,-0.296112,-0.575835,-0.724368,-13.244202,0.018445,2.347826,54.0,1.525001e+09,1.520259e+09,239.0,10.0,0.260870,6.0,7.0,1.0,-179.212942,22.0,C_ID_92a2005557,-0.820283
1,356.0,6.0,723.0,0.010712,0.011385,0.010283,0.968571,339,350,0.088571,31,0.074568,-0.295163,7,5.0,24,10,1.551429,-1,543,2.282448,57,142,11.034286,0,-5.031429,-12,14.477519,12,4.630299,-0.600018,-0.742400,-210.006336,0.148200,1.114286,390,1.517438e+09,1.483720e+09,327,24,0.377143,132,50,2,1.0,6.0,0.000000,0.0,-0.550160,-0.606486,4.0,396.0,5.0,1.0,1.000000,1.0,6.0,0.000000,5.0,6.0,11.000000,2.0,1.500000,1.0,0.300000,2.0,-0.701858,-0.725956,-0.739410,-4.355735,0.000205,9.333333,56.0,1.522393e+09,1.517505e+09,270.0,4.0,0.000000,0.0,4.0,1.0,-214.362071,52.0,C_ID_3d0044924f,0.392913
2,44.0,0.0,876.0,0.010610,0.008752,0.010283,0.953488,41,43,0.000000,0,-0.087803,0.358458,7,163.0,14,0,0.000000,0,0,0.000000,8,13,10.023256,0,-8.604651,-13,14

In [19]:
train_test.set_index(key, inplace=True)
best_pred = best_pred.join(train_test)
best_pred.to_csv(f'../output/{start_time[4:11]}_elo_exp113.csv', index=True)